In [ ]:
from WPI_SCA_LIBRARY.CWScope import *
from WPI_SCA_LIBRARY.LeakageModels import *
from WPI_SCA_LIBRARY.Metrics import *
firmware_path = "C:\\Users\\samka\\PycharmProjects\\MQP\\SCLA_API_MQP\\WPI_SCA_LIBRARY\\firmware\\simpleserial-aes-CWLITEARM-SS_2_1.hex"

## Create A File Using The Custom File Framework
We can create the parent directory and create the basic structure by calling the constructor for the `FileParent` class by simply specifying the file name and path.

In [ ]:
# Let's create a file called NEHWS Demo and save it in the same directory that we currently in!
file = FileParent(name="NEHWS_Demo", path="C:\\Users\\samka\\PycharmProjects\\MQP\\SCLA_API_MQP\\Demos", existing=False)

## Collect some data from the ChipWhisperer-Lite Using the Oscilloscope Interface
We can collect some traces from a ChipWhisperer target board running AES128. This uses the most simple capture procedure supported leaving most settings to their default values. 

In [ ]:
# We will load the target board with a simple AES implementation and collect 1000 traces each with 3000 samples. We will use a built-in key, text pair generator however you can specify custom values as an array if you want.
scope = CWScope(firmware_path, gain=25, num_samples=3000, offset=0, target_type=cw.targets.SimpleSerial2, target_programmer=cw.programmers.STM32FProgrammer)
traces, keys, plaintexts, ciphertexts = scope.standard_capture_traces(num_traces=1000, fixed_key=True, fixed_pt=False)

## Create an experiment and populate it with a few datasets
We collected some power traces along with keys, plaintexts, and ciphertexts. Let's create and experiment and add these as datasets to our file!

In [ ]:
experiment_1 = file.add_experiment(name="Experiment1")
traces_dataset = experiment_1.add_dataset(name="Traces Dataset", data_to_add=traces, size=(1000, 3000), datatype='float32')